# <font color=#cd0000> Propósito principal </font>
- La idea de este librillo es preparar el entorno para realizar pruebas para cualquier DataSet a ser clasificado por cualquier codificación aplicada a RF

## <font color=#cd0000> Leyenda </font>
- Los apartados titulados con el código de colores HEX: `#cd0000` serán apartados que tendrán todos los librillos, en concreto, aquellos especificados en el apartado `Síntesis de los criterios usados` del trabajo.
- Los apartados titulados con el código de colores HEX: `#2451ff` serán apartados de conclusiones propias de este librillo resultado de aplicar un estudio personalizado para cada planteamiento.

# <font color=#cd0000> Prerrequisitos </font>
## <font color=#cd0000> Entorno de ejecución </font>
- Cambiamos el directorio raíz del librillo para acceder cómodamente a las funciones de utilidad.

In [ ]:
import os

os.chdir('../..')
os.listdir()


## <font color=#cd0000> Constantes y variables predefinidas </font>
- TODO -> Añadir SEED a todas las particiones.

In [ ]:
HEARTBEAT_PATH = "data/heartbeat"
EPILEPSY_PATH = "data/epilepsy"
SEGUIMIENTO_OCULAR_PATH = "data/seguimiento-ocular/Data/Hospital"
SEGUIMIENTO_OCULAR_FOLDERS_ID = range(1, 12+1)

SEED = 1

# <font color=#cd0000> Carga del Dataset </font>
- TODO: Breve descripción

In [ ]:
# TODO - Change with known data
from utils.load_data import import_epilepsy_dataset

# train, test = import_heartbeat_dataset(HEARTBEAT_PATH)
# all_data = Data(Data.concat_data(
#     train.original_data, test.original_data))
# pickle.dump(all_data, open("HeartBeat_tmp_data.pkl", 'wb'))


In [ ]:
import pickle

all_data = pickle.load(open("HeartBeat_tmp_data.pkl", 'rb'))

In [ ]:
all_data.reset_changes()


# <font color=#cd0000> Preprocesamiento </font>

## <font color=#cd0000> Eliminación de datos inválidos y valores atípicos </font>
- TODO: Breve descripción de qué es un dato inválido (-1's en columna, etc.)
- Eliminaremos aquellos valores fuera de los percentiles 5 y 95.
- TODO: Definiremos cuál será el límite de outliers permitido por serie temporal

In [ ]:
# TODO - Remove invalid data

In [ ]:
all_data.remove_outliers(
    headers=all_data.get_derived_data_columns()['attrs'],
    outliers_limit=.3
)


In [ ]:
import pandas as pd

# Remaining series
print("Previous number of series: {}".format(
    len(pd.unique(all_data.original_data['id']))))
print("Actual number of series: {}".format(
    len(pd.unique(all_data.derived_data['id']))))


## <font color=#cd0000> Resoluciones a aplicar </font>
- TODO:
  - Si las series son rápidas (muchos cambios en poco tiempo) especificar resoluciones altas (sin modificaciones).
  - Si las series son lentas (pocos cambios en mucho tiempo) especificar resoluciones bajas (eliminamos datos).

In [ ]:
# Series lentas
all_data.reduce_sampling_rate(remove_one_each_n_samples=2)
all_data.derived_data

# <font color=#cd0000> Codificación </font>
- TODO: Breve descripción de la codificación

# <font color=#cd0000> División en ventanas </font>
- Solo aplicaremos enventanado si no ha sido aplicado anteriormente
- TODO: Especificar tamaño de ventana esperado como mejor y adjuntar otro tamaño de ventana para comparar (al menos 2 más)
- TODO: No es necesario aplicar siempre el enventanado, revisar análisis en profundidad.

In [ ]:
# Estudiamos eventos globales (series lentas)
ws_x = all_data.get_shortest_serie().shape[0]
all_data_ws_x, all_data_windows_per_serie_x = \
    all_data.split_into_windows(all_data.derived_data, window_size=ws_x)

# Estudiamos eventos locales (series rápidas)
ws_y = int(all_data.get_shortest_serie().shape[0]/2)
all_data_ws_y, all_data_windows_per_serie_y =\
    all_data.split_into_windows(all_data.derived_data, window_size=ws_y)

In [ ]:
from utils.data_extraction import Data

all_data_large_windows = Data(all_data_ws_x, all_data_windows_per_serie_x)
all_data_short_windows = Data(all_data_ws_y, all_data_windows_per_serie_y)

## <font color=#cd0000> Codificación sobre las ventanas </font>

In [ ]:
from utils.codifications import temporal_trend_fn

all_data_large_windows.apply_codifications([temporal_trend_fn])

## <font color=#cd0000> Partición de los datos </font>

In [ ]:
from utils.data_extraction import Data

X_train_Data, X_test_Data, y_train, y_test = all_data.train_test_split(
    criterion='windowed',
    train_size=.8,
    random_state=SEED,
    drop_columns=[]
)

X_train_Data = Data(X_train_Data)
X_test_Data = Data(X_test_Data)


# <font color=#cd0000> Diseño de la topología del bosque </font>
- Número de estimadores inicial recomendado
- Profundidad máxima recomendada

## <font color=#cd0000> Preparación de los datos </font>

In [ ]:
X_train = X_train_Data.derived_data.drop(['id', 'class'], axis=1)
X_test = X_test_Data.derived_data.drop(['id', 'class'], axis=1)

## <font color=#cd0000> Entrenamiento </font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=SEED)
clf.fit(X_train, y_train)

## <font color=#cd0000> Clasificación </font>

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf.predict(X_test)
y_true = np.asarray(y_test)
    
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, zero_division=0))


## <font color=#cd0000> Discusión y conclusiones </font>
- TODO

# <font color=#cd0000> Randomized Search </font>
- Búsqueda de hiper-parámetros aleatoria con RF maximizando ``macro avg f1-score``

## <font color=#cd0000> Rangos de búsqueda </font>
- Como vimos anteriormente los rangos de búsqueda aleatoria de los mejores hiper-parámetros serán los siguientes

In [ ]:
N_ESTIMATORS_RANGE = TODO
MAX_DEPTH_RANGE = TODO

In [ ]:
import random
import pickle
import utils.constants as cs
from sklearn.ensemble import RandomForestClassifier
from utils.classifier_utils import (windowed_cross_val,
                                    compute_classification_reports_means)
from utils.plot_utils import pretty_print_classification_report_dict

PKL_DIR = 'pkl/RF/<dataset>/'


def rf_randomized_search_cv(
        windowed_series,
        relation_with_series,
        prefix,
        cv=5):
    global PKL_DIR
    all_clf_used = {}

    n_samples = 5
    n_estimators_list = random.sample(list(N_ESTIMATORS_RANGE), n_samples)
    max_depth_list = random.sample(list(MAX_DEPTH_RANGE), n_samples)

    best_hyp_params = None
    best_score = 0
    for n_estimators in n_estimators_list:
        for max_depth in max_depth_list:
            clf = RandomForestClassifier(
                n_estimators=n_estimators,
                max_depth=max_depth,
                random_state=SEED
            )

            reports = windowed_cross_val(
                clf,
                windowed_series,
                relation_with_series,
                estimator_type=cs.ESTIMATOR_SKLEARN,
                cv=cv,
                drop_columns=['id', 'class'],
                seed=SEED
            )
            mean_report = compute_classification_reports_means(reports)
            all_clf_used[(n_estimators, max_depth)] = mean_report

            if mean_report['macro avg']['f1-score'][0] >= best_score:
                best_score = mean_report['macro avg']['f1-score'][0]
                best_hyp_params = (n_estimators, max_depth)
                best_report = mean_report

            print("\t\t--------------ACTUAL BEST: N_Estimators={}; Max_Depth={}--------------"
                  .format(best_hyp_params[0], best_hyp_params[1]))
            pretty_print_classification_report_dict(best_report)
            print("\t\t--------------ITERATION: N_Estimators={}; Max_Depth={}--------------"
                  .format(n_estimators, max_depth))
            pretty_print_classification_report_dict(mean_report)

    with open(PKL_DIR + prefix, 'wb') as file:
        pickle.dump(all_clf_used, file)

    return best_hyp_params, best_report


In [ ]:
PKL_NAME = "rf_sample.pkl"

rf_randomized_search_cv(
    X_train_Data.derived_data,
    X_train_Data.derived_data_windows_per_serie,
    PKL_NAME,
    cv=5)


# <font color=#cd0000> Randomized Search con múltiples ejecuciones en lugar de Validación Cruzada </font>
- Solo si tenemos pocos datos
- Ejecutaremos el mismo modelo sobre diferentes particiones del conjunto de datos original para observar su desempeño.

In [ ]:
# TODO

# <font color=#cd0000> Análisis de resultados </font>
- Según la búsqueda aleatoria de hiper-parámetros, la mejor combinación, es la de ``n_estimators`` = TODO y ``max_depth`` = TODO:
    ```
        TODO
    ```
- Ahora vamos a visualizar la evolución de los resultados (25 resultados) para observar cómo avanza nuestra métrica objetivo -> Macro Average F1-Score.

In [ ]:
import pickle
from utils.plot_utils import plot_score

all_reports = pickle.load(open(PKL_DIR + PKL_NAME, 'rb'))

In [ ]:
all_reports

{(49,
  168): "defaultdict(<function compute_classification_reports_means.<locals>.<lambda> at 0x0000025CB7C845E0>, {'accuracy': (0.7233689284324996, 0.05061900029750596), 'abnormal': defaultdict(<function compute_classification_reports_means.<locals>.<lambda>.<locals>.<lambda> at 0x0000025CBD425CA0>, {'precision': (0.7541104422971636, 0.06743790753482802), 'recall': (0.9270811701454365, 0.015901107315767472), 'f1-score': (0.8295537595629229, 0.0375325340349416), 'support': (18954.0, 1566.465448070911)}), 'normal': defaultdict(<function compute_classification_reports_means.<locals>.<lambda>.<locals>.<lambda> at 0x0000025CBD425D30>, {'precision': (0.4509960817786235, 0.10228114578319143), 'recall': (0.16701861959832973, 0.02201700921372155), 'f1-score': (0.23821859885908836, 0.01737480123923299), 'support': (6885.0, 1899.618382728489)}), 'macro avg': defaultdict(<function compute_classification_reports_means.<locals>.<lambda>.<locals>.<lambda> at 0x0000025CBD425DC0>, {'precision': (0.60

# <font color=#cd0000> Evaluación sobre el conjunto de validación </font>
- Vamos a llevar a cabo la evaluación final sobre el conjunto de validación (esto es lo que irá al apartado de ``Pruebas y Resultados`` de la memoria).

## <font color=#cd0000> Preparación de los datos </font>

In [ ]:
X_train = X_train_Data.derived_data.drop(['id'], axis=1)
X_test = X_test_Data.derived_data.drop(['id'], axis=1)

## <font color=#cd0000> Entrenamiento </font>

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(n_estimators=TODO, max_depth=TODO, random_state=SEED)
clf.fit(X_train, y_train)


RandomForestClassifier(max_depth=200, n_estimators=75, random_state=1)

## <font color=#cd0000> Clasificación </font>

In [ ]:
import numpy as np
from sklearn.metrics import confusion_matrix, classification_report

y_pred = clf.predict(X_test)
y_true = np.asarray(y_test)
    
print(confusion_matrix(y_true, y_pred))
print(classification_report(y_true, y_pred, zero_division=0))


[[18810  1440]
 [ 9943  1397]]
              precision    recall  f1-score   support

    abnormal       0.65      0.93      0.77     20250
      normal       0.49      0.12      0.20     11340

    accuracy                           0.64     31590
   macro avg       0.57      0.53      0.48     31590
weighted avg       0.60      0.64      0.56     31590



# <font color=#cd0000> Conclusiones </font>
- TODO - Unas breves conclusiones sobre los resultados obtenidos (influencia de la codificación, ...)